In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import keras.backend as K
from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM

In [ ]:
train = pd.read_csv('/Users/dimamuzalevskiy/desktop/all/train_1.csv').fillna(0)
df = train.transpose()

df.columns = df.iloc[0]
df = df[1:]
y = []

for i in range(1,df.shape[1]):    
    l = df.iloc[0:,i]   
    y.append(list(map(int,l)))

In [ ]:
df.head()

In [ ]:
def network():

    in_out_neurons = 40 
    hidden_neurons = 200

    model = Sequential()
    model.add(LSTM(hidden_neurons, return_sequences=False, input_shape=(None, in_out_neurons)))
    model.add(Dense(in_out_neurons, input_dim=hidden_neurons))  
    model.add(Activation("linear")) 
    model.compile(optimizer="Adam", loss=smape_loss) 
    
    return model

In [ ]:
def smape_loss(y_true, y_pred):
    
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true) + K.abs(y_pred), K.epsilon(), None))
    
    return 200 * K.mean(diff, axis=-1)

In [ ]:
model=network()
model.summary()

In [ ]:
def input_data(data, look = 3):  

    X = []
    Y = []
    
    for i in range(len(data)-look):
        
        X.append(data.iloc[i:i+look].as_matrix())
        Y.append(data.iloc[i+look].as_matrix())
        
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

In [ ]:
def train_test_split(df, test_size=0.2):  

    n = round(len(df) * (1 - test_size))

    X_train, y_train = input_data(df.iloc[0:n])
    X_test, y_test = input_data(df.iloc[n:])

    return (X_train, y_train), (X_test, y_test)

In [ ]:
values = df.iloc[:,:40]
values.head()

In [ ]:
(X_train, y_train), (X_test, y_test)=train_test_split(values)

In [ ]:
X_train.shape,X_test.shape, y_train.shape,y_test.shape

In [ ]:
model=network()
model.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.05)

In [ ]:
predicted = model.predict(X_test)
error =(((np.abs(y_test - predicted)) /(np.abs(y_test) + np.abs(predicted))) * 200).mean(axis=0)
error.mean()

In [ ]:
predicted.shape

In [ ]:
series = []

for i in range(0,df.shape[1],40):
    
    series.append(i)

In [ ]:
models = []
result = []

In [ ]:
for i in range(1,len(series)):
    
    values = df.iloc[:,series[i-1]:series[i]]
    (X_train, y_train), (X_test, y_test)=train_test_split(values)
    model = network()
    model.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.05)
    predicted = model.predict(X_test)
    error =(((np.abs(y_test - predicted)) /(np.abs(y_test) + np.abs(predicted))) * 200).mean(axis=0)
    result.append(error.mean())
    models.append(model)